In [ ]:
import requests
from urllib.parse import urlparse, parse_qs
import json
import polyline
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import geocoder
import numpy as np
import folium
import base64
import os
import time
from faker import Factory

In [ ]:
# To generate an authorization_code, the user must authenticate with strava through the app using the OAuth 2.0 prottocol. 
# If you don't want to set up a web server, the athelete can put the below url in their browser. 
# Be sure to replace ['client_id'] with the app's client ID found at https://www.strava.com/settings/api

# https://www.strava.com/oauth/authorize?client_id=[client_id]&redirect_uri=http://localhost&response_type=code&grant_type=authorization_code&scope=activity:read_all

# The Athlete will be directed to a strava athentication page asking to give the app permission to their profile
# After clicking 'Authorize' the user will be redirected and the new url will contain the authorization code 
# Set the response variable below to the redirect url, it should look like this 'http://localhost/?state=&code=[your_code_here]&scope=read,activity:read_all'
    
redirect = ''

# Pasre the redirect url and set the authorization code to a variable that can be used in the request
# Using the modules urlparse and parse_qs from the urllib library, the below code parses the redirect url, queries for the authorization code and sets it to the variable `authorization_code`

parsed_url = urlparse(redirect)
authorization_code = parse_qs(parsed_url.query)['code'][0]
print(authorization_code)

# Make a request to the Strava API for the Athlete's token with the scope necessary
# But first, set the necessary variables to make the call: https://www.strava.com/settings/api

token_url ='https://www.strava.com/api/v3/oauth/token'
client_id =  # This is the client id for your app
client_secret = '' # This is the secret code for your app, keep it safe
grant_type = 'authorization_code'

# Once the variables are set above, make the post request and set it to the variable token_response
token_response = requests.post(token_url, data={'client_id':client_id,'client_secret':client_secret,'code':authorization_code,'grant_type':grant_type})


In [ ]:
# Convert the response to json and retrive the token
json_token_response = json.loads(token_response.text)

# Call the keys of the desired values and set them as variables
access_token = json_token_response['access_token']
athelete_id = json_token_response['athlete']['id']

print(athelete_id)
print(access_token)


In [ ]:
# Now request the athelete's saved routes
# But what if they have A LOT of saved routes? 
# We can create a function to make the request and call it in a for loop to iterate through multiple pages of routes

# get_routes function that takes access token, results per page, and page number
# 200 is the default for per_page as it is the maximum number of routes that can be returned

def get_routes(access_token, athelete_id, per_page=200, page=1):
    routes_url = 'https://www.strava.com/api/v3/athletes/' + str(athelete_id) + '/routes'
    data = {'access_token':access_token, 'per_page':per_page, 'page':page}
    
    response = requests.get(
        routes_url,
        data=data
    )
    return response

# Now we can iterate through our routes catalog
# Since each request returns a list of json results, we'll nest another for loop to pull each route and load them to our list individually
max_pages = 3
data = []
for page_number in range(1, max_pages + 1):
    page_data = json.loads(get_routes(access_token, athelete_id, page=page_number).text)
    for route in page_data:
        data.append(route)
    if page_data == []:
        break

In [ ]:
# Create a new list of dictionaries filtered with only the details needed and in a more useful format

# Start by initializing route_list
route_list = []

# Then iterate over the data variable using a for loop to build dictionaries that are then appended to route_list
for route in data:
    route_details = {}
    route_details['id'] = route['id']
    route_details['name'] = route['name']
    route_details['description'] = route['description']
    route_details['timestamp'] = route['timestamp']
    route_details['estimated_moving_time'] = route['estimated_moving_time']
    route_details['distance'] = round((route['distance'] / 1000),2)
    route_details['elevation_gain'] = round(route['elevation_gain'], 2)
    route_details['summary_polyline'] = route['map']['summary_polyline']
    route_list.append(route_details.copy())

    
# Now save it as a pandas DataFrame
routes_df = pd.DataFrame(route_list)   
    

In [ ]:
# Apply polyline.decode() to the summary_polyline and store the resulting list of lat/long tuples to the column `polyline` in the DataFrame
routes_df['polyline'] = routes_df['summary_polyline'].apply(polyline.decode)

In [ ]:
# Initialize my_route with the first route in the routes DataFrame
my_route = routes_df.iloc[0,:]

# define the centroid (i.e. center position of the route where the map should focus) 
centroid = [
    np.mean([coord[0] for coord in my_route['polyline']]),
    np.mean([coord[1] for coord in my_route['polyline']])
]

# Use folium library to plot my_route on the map 
m = folium.Map(location=centroid, zoom_start=12)
folium.PolyLine(my_route['polyline'], color='red').add_to(m)
display(m)

In [ ]:
# Using list comprehension, make a list of starting coordinates for each route
start_latlong = [route[0] for route in routes_df['polyline']]

# Initialize start_state list
start_state = []

# Loop through the starting coordinates and translate them to a state using reverse geocoding
for coord in start_latlong:
    g = geocoder.arcgis([coord[0], coord[1]], method='reverse').state
    start_state.append(g)
    
# Now add the start_state to our DataFrame as a column
routes_df['state'] = start_state

# Create a mask to filter for only routes starting in California
ca_mask = routes_df['state'] == 'California'

# Call the California mask against the original DataFrame and save a copy to a new DataFrame so there's no reference
ca_routes_df = routes_df[ca_mask].copy()


In [ ]:
# Write a function that accepts latitude and longitude and returns the corresponding elevation
def get_elevation (lat, long):
    url_endpoint = 'https://api.open-elevation.com/api/v1/lookup'
    parameters = {'locations': f'{lat}, {long}'}
    response = requests.get(url_endpoint,params=parameters).json()['results'][0]
    return response['elevation']    


In [ ]:
# Now, using the index of the DataFrame, iterate through each record and generate elevation values for each route's polyline coordinates
# Save the responses as a list of lists

elevations = []
for idx in tqdm(ca_routes_df.index):
    route = ca_routes_df.loc[idx, :]
    # List Comprehension calling our function
    elevation = [get_elevation(coord[0],coord[1]) for coord in route['polyline']]
    elevations.append(elevation)

# Add the list of elevation lists to the DataFrame
ca_routes_df['elevation'] = elevations

In [ ]:
# Initialize elevation_profile dictionary
elevation_profile = {}

# Loop through the DataFrame
for row in ca_routes_df.iterrows():
    # Ignore the index, set the data Series to row_values 
    row_values = row[1]
    
    # Create a figure, plotting the moving average of the elevation seris and save it as a .png file
    fig, ax = plt.subplots(figsize=(6,2))
    ax = pd.Series(row_values['elevation']).rolling(3).mean().plot(
        ax=ax,
        color='red',
        legend=False
    )
    ax.set_ylabel('Elevation')
    ax.axes.xaxis.set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    png = 'elevation_profile_{}.png'.format(row_values['id'])
    fig.savefig(png, dpi=75)
    plt.close()
    
    # read the saved png file and set it to the elevation profile dictionary with its corresponding route id
    elevation_profile[row_values['id']] = base64.b64encode(open(png, 'rb').read()).decode()
    
    #delete file
    os.remove(png)
    

In [ ]:
# Define the map resolutiion and size
resolution, width, height = 75, 6, 6.5

# Define a function to find centroid of all routes combined
def centroid(polylines):
    x, y = [], []
    for polyline in polylines:
        for coord in polyline:
            x.append(coord[0])
            y.append(coord[1])
    return [(min(x)+max(x))/2, (min(y)+max(y))/2]


# Generate the folium map
m = folium.Map(location=centroid(ca_routes_df['polyline']), zoom_start=8)

#iterate through routes, plotting each on the map and generating their details popup
for row in ca_routes_df.iterrows():
    fake = Factory.create()
    color = fake.hex_color()
    row_index = row[0]
    row_values = row[1]
    folium.PolyLine(row_values['polyline'], color=color).add_to(m)

    # Halfway coords for popup
    halfway_coord = row_values['polyline'][int(len(row_values['polyline'])/2)]


    
    # Popup text
    html = """
    <h3>{}</h3>
        <p>
            <code>
            Description : {} 
            </code>
        </p>
    <h4>Details</h4>
        <p> 
            <code>
                Distance&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp: {:.2f} km <br>
                Elevation Gain&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp: {} m <br>
                Est. Moving Time&nbsp;&nbsp;&nbsp;&nbsp: {} <br>
                https://www.strava.com/routes/{} <br>
            </code>
        </p>
    <img src="data:image/png;base64,{}">
    """.format(
        row_values['name'], 
        row_values['description'],  
        row_values['distance'], 
        row_values['elevation_gain'], 
        time.strftime('%H:%M:%S', time.gmtime(row_values['estimated_moving_time'])),  
        row_values['id'], 
        elevation_profile[row_values['id']], 
    )
    
    # add marker to map
    iframe = folium.IFrame(html, width=(width*resolution)+20, height=(height*resolution)+20)
    popup = folium.Popup(iframe, max_width=2650)
    icon = folium.Icon(color='white', icon_color = color, icon='map-pin', prefix='fa')
    marker = folium.Marker(location=halfway_coord, popup=popup, icon=icon)
    marker.add_to(m)

m.save('mymap.html')
display(m)